In [3]:
import re
import os
import pandas as pd

# from google.colab import drive  # Colab 드라이브 마운트
# drive.mount('/content/drive', force_remount=True)

BASE_PATH    = ''  # 드라이브 내 작업 경로
IN_PATH = os.path.join(BASE_PATH,"여성맞춤정책_요약_2차_결과_요약수정_최종.xlsx")

def fix_bullet_lines(text: str) -> str:
    lines = text.splitlines()
    fixed = []
    for line in lines:
        m = re.match(r'^(\s*)-(\s*)(.*)$', line)
        if m:
            content = m.group(3)
            fixed.append(f" - {content.strip()}")
        else:
            fixed.append(line)
    return "\n".join(fixed)

def fix_parentheses_spacing(text: str) -> str:
    s = text

    # --- [A] 숫자만 들어있는 괄호는 플레이스홀더로 대체 (내부는 붙여쓰기 유지) ---
    placeholder_fmt = "<<<P_NUM:{}>>>"
    nums = []

    def _num_hold(m):
        num = m.group(1)
        idx = len(nums)
        nums.append(num)
        return placeholder_fmt.format(idx)

    s = re.sub(r'\(\s*(\d+)\s*\)', _num_hold, s)

    # --- [B] 일반 괄호 내부 공백 정리: "(내용)" -> "( 내용 )" ---
    s = re.sub(r'\(\s*([^\)]*?)\s*\)', r'( \1 )', s)

    # --- [C] 괄호 외부 공백 정리(일반 괄호에만 적용됨) ---
    s = re.sub(r'(?P<prev>\S)\(', r'\g<prev> (', s)  # 앞쪽
    s = re.sub(r'\)(?P<next>\S)', r') \g<next>', s)  # 뒤쪽

    # --- [D] 숫자 괄호 복원 ---
    def _num_restore(m):
        idx = int(m.group(1))
        return f'({nums[idx]})'
    s = re.sub(r'<<<P_NUM:(\d+)>>>', _num_restore, s)

    # --- [E] 숫자 괄호의 외부 공백 보정 (복원 후 적용) ---
    # 앞쪽: 붙은문자(숫자/한글/영문/%) + "(숫자)" -> "문자 (숫자)"
    s = re.sub(r'(?P<prev>\S)\((\d+)\)', r'\g<prev> (\2)', s)
    # 뒤쪽: "(숫자)문자" -> "(숫자) 문자"
    s = re.sub(r'\((\d+)\)(?P<next>\S)', r'(\1) \g<next>', s)

    # --- [F] 라인별 다중 공백 축소(선행 들여쓰기는 유지) ---
    lines = []
    for line in s.splitlines():
        line = line.replace('\t', ' ')
        m = re.match(r'^(\s*)(.*)$', line)
        lead, body = m.group(1), m.group(2)
        body = re.sub(r' {2,}', ' ', body)
        lines.append(lead + body)
    return "\n".join(lines)

def process_cell(val):
    if pd.isna(val):
        return val
    t = str(val)
    t = fix_bullet_lines(t)
    t = fix_parentheses_spacing(t)
    return t

# 모든 시트/모든 열 처리
xls = pd.read_excel(IN_PATH, sheet_name=None)
out_sheets = {}
for sheet_name, df in xls.items():
    df2 = df.copy()
    for col in df2.columns:
        if df2[col].dtype == object:
            df2[col] = df2[col].map(process_cell)
    out_sheets[sheet_name] = df2

# Google Drive 경로로 저장
OUT_PATH = os.path.join(BASE_PATH, "여성맞춤정책_요약_2차_결과_정리.xlsx")

with pd.ExcelWriter(OUT_PATH, engine="xlsxwriter") as writer:
    for sheet_name, df in out_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# df.head(3)

In [4]:
import pandas as pd

# 파일 경로 설정
base_path = r"C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_정리.xlsx"
merge_path = r"C:\Users\hyunj\Women-s-Government-Aid\UI\정책큐레이션_통합데이터_v1.0.xlsx"

# 엑셀 파일 불러오기
base_df = pd.read_excel(base_path)
merge_df = pd.read_excel(merge_path)

# 병합 기준 열 설정 (1~4열: 인덱스 기준 0~3)
key_columns = base_df.columns[:4].tolist()

# 병합 수행 (base_df 기준으로 left join)
merged_df = pd.merge(base_df, merge_df, on=key_columns, how='left')

# 결과 저장
output_path = "여성맞춤정책_요약_2차_결과_병합_최종.xlsx"
merged_df.to_excel(output_path, index=False)

print("병합 완료! 결과 파일 저장 위치:", output_path)


병합 완료! 결과 파일 저장 위치: 여성맞춤정책_요약_2차_결과_병합_최종.xlsx


In [6]:
import pandas as pd

# 병합 파일 경로
PATH_MERGED = r"C:\Users\hyunj\Women-s-Government-Aid\UI\여성맞춤정책_요약_2차_결과_요약수정_최종.xlsx"

# 기준 열 정의
KEY_COLS = ["대상유형", "지역", "제목", "detail_url", "신청기간", "신청방법", "접수기관", "지원형태", "지원대상", "지원내용"]

# 파일 불러오기
df = pd.read_excel(PATH_MERGED)

# 문자열 정제 함수
def clean_str(x):
    if pd.isna(x):
        return ""
    return str(x).strip().lower().replace("\n", "").replace("\r", "").replace(" ", "")

# 기준 열 정제
for col in KEY_COLS:
    if col in df.columns:
        df[col] = df[col].apply(clean_str)

# row_key 생성
df["row_key"] = df[KEY_COLS].agg("|".join, axis=1)

# 중복 제거 (첫 번째 행만 남김)
df_unique = df.drop_duplicates(subset=["row_key"], keep="first").drop(columns=["row_key"])

# 결과 저장 (선택)
df_unique.to_excel("여성맞춤정책_요약_병합_최종.xlsx", index=False)

# 결과 확인
print(f"원래 행 수: {len(df)}")
print(f"유니크 행 수: {len(df_unique)}")


원래 행 수: 1292
유니크 행 수: 1292
